In [3]:
import sys
# 1. Uninstall the breaking versions
!{sys.executable} -m pip uninstall -y mediapipe protobuf

# 2. Install the compatible 2022 stack (MediaPipe 0.8.11 + Protobuf 3.19)
!{sys.executable} -m pip install mediapipe==0.8.11 protobuf==3.19.6

Found existing installation: mediapipe 0.10.21
Uninstalling mediapipe-0.10.21:
  Successfully uninstalled mediapipe-0.10.21
Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3


ERROR: Could not find a version that satisfies the requirement mediapipe==0.8.11 (from versions: 0.10.5, 0.10.7, 0.10.8, 0.10.9, 0.10.10, 0.10.11, 0.10.13, 0.10.14, 0.10.18, 0.10.20, 0.10.21)
ERROR: No matching distribution found for mediapipe==0.8.11


In [ ]:
import sys

# 1. Install MediaPipe but IGNORE dependencies (so it doesn't break Protobuf)
print("⬇️ Force Installing MediaPipe (No Dependencies)...")
!{sys.executable} -m pip install mediapipe --no-deps

# 2. Verify we have the correct "Golden Combo"
# We want: MediaPipe (Any) + Protobuf (3.20.3)
import google.protobuf
try:
    import mediapipe as mp
    print(f"✅ Success! MediaPipe Version: {mp.__version__}")
except ImportError:
    print("⚠️ MediaPipe installed but needs restart.")

print(f"✅ Protobuf Version: {google.protobuf.__version__} (Should be 3.20.3)")
print("⚠️ NOW: Restart the Kernel one last time! ⚠️")

In [ ]:

import google.protobuf
print(f"✅ Fixed Protobuf Version: {google.protobuf.__version__}")


In [ ]:
import os
import sys

# ==========================================
# 1. CRITICAL SETUP (Must be first)
# ==========================================
# Force Python Protobuf (Prevents the 'GetMessageClass' crash)
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# Force CPU (Prevents Windows GPU freeze)
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# ==========================================
# 2. THE GOLDEN IMPORT ORDER
# ==========================================
print("⏳ Importing Libraries (Order Matters)...")

import cv2

# IMPORT MEDIAPIPE FIRST (Before TensorFlow!)
import mediapipe as mp

# IMPORT TENSORFLOW SECOND
import tensorflow as tf
import pandas as pd
import numpy as np
from collections import deque

print("✅ Libraries Imported Successfully.")

# ==========================================
# 3. CONFIGURATION
# ==========================================
MLP_PATH = 'arsl_mediapipe_mlp_model_best.h5'      
LABELS_PATH = 'arabic_final_training_data.csv'     

# ==========================================
# 4. LOAD MODEL & DATA
# ==========================================
if os.path.exists(MLP_PATH):
    # compile=False is safer for inference
    model = tf.keras.models.load_model(MLP_PATH, compile=False)
    print("✅ Arabic Model Loaded!")
else:
    raise SystemExit(f"❌ Missing Model: {MLP_PATH}")

if os.path.exists(LABELS_PATH):
    df = pd.read_csv(LABELS_PATH)
    labels = sorted(df['label'].unique())
    print(f"✅ Loaded {len(labels)} Classes (Including Space/Del/Nothing)")
else:
    raise SystemExit(f"❌ Missing Labels: {LABELS_PATH}")

# ==========================================
# 5. MAIN APP LOOP
# ==========================================
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)

sentence = []
history = deque(maxlen=5)
current_text = "..."

print("\n📸 Opening Camera... (Press 'q' to Quit)")
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("❌ Camera disconnected.")
        break
    
    # Preprocessing
    frame = cv2.flip(frame, 1)
    h, w, c = frame.shape
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process Hand (This is where it crashed before)
    results = hands.process(img_rgb)
    
    predicted_char = "nothing" 
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Extract Skeleton
            row = []
            for lm in hand_landmarks.landmark:
                row.extend([lm.x, lm.y, lm.z])
            X = np.array([row], dtype='float32')
            
            # Predict
            pred = model.predict(X, verbose=0)[0] 
            class_idx = np.argmax(pred)
            prob = np.max(pred)
            
            if prob > 0.80:
                predicted_char = labels[class_idx]
            else:
                predicted_char = "nothing"
        
        history.append(predicted_char)
    else:
        history.append("nothing")

    # Smoothing & Typing Logic
    if len(history) == 5:
        most_common = max(set(history), key=history.count)
        if history.count(most_common) > 3:
            if most_common != current_text:
                current_text = most_common
                if current_text == "space":
                    sentence.append(" ")
                elif current_text == "del":
                    if len(sentence) > 0: sentence.pop()
                elif current_text != "nothing":
                    sentence.append(current_text)

    # UI
    color = (0, 200, 0) if results.multi_hand_landmarks else (50, 50, 50)
    cv2.rectangle(frame, (0, 0), (w, 80), color, -1)
    cv2.putText(frame, f"Sign: {current_text}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    
    cv2.rectangle(frame, (0, h-60), (w, h), (0, 0, 0), -1)
    cv2.putText(frame, "".join(sentence), (20, h-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    cv2.imshow('Arabic Sign Language Translator', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()